In [ ]:
!pip install tensorflow opencv-python matplotlib numpy pandas scikit-learn kaggle shap reportlab streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [3]:
from google.colab import files
import os

# Upload kaggle.json file
files.upload()

# Move to the correct directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d lukechugh/best-alzheimer-mri-dataset-99-accuracy
!unzip best-alzheimer-mri-dataset-99-accuracy.zip -d dataset


Streaming output truncated to the last 5000 lines.
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1106).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1107).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1108).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1109).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (111).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1110).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1111).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1112).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1113).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1114).jpg  
  inflating: dataset/Combined Dataset/train/No Impairment/NoImpairment (1115).jpg  
  inflating: dataset/Combi

In [4]:
import os
import zipfile

# Define dataset path
zip_path = "/content/best-alzheimer-mri-dataset-99-accuracy.zip"
extract_path = "/content/dataset"

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [5]:
print("Available directories:", os.listdir(extract_path))


Available directories: ['Combined Dataset']


In [6]:
data_path = "/content/dataset/Combined Dataset"
print("Dataset folders:", os.listdir(data_path))


Dataset folders: ['test', 'train']


In [7]:
train_path = "/content/dataset/Combined Dataset/train"
test_path = "/content/dataset/Combined Dataset/test"

# Verify folders
print("Train folders:", os.listdir(train_path))
print("Test folders:", os.listdir(test_path))


Train folders: ['Moderate Impairment', 'Very Mild Impairment', 'Mild Impairment', 'No Impairment']
Test folders: ['Moderate Impairment', 'Very Mild Impairment', 'Mild Impairment', 'No Impairment']


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
img_size = 224
batch_size = 32

# Data Augmentation & Normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% Train, 20% Validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load Train Data
train_data = datagen.flow_from_directory(
    train_path, target_size=(img_size, img_size),
    batch_size=batch_size, class_mode='categorical', subset='training'
)

# Load Validation Data
val_data = datagen.flow_from_directory(
    train_path, target_size=(img_size, img_size),
    batch_size=batch_size, class_mode='categorical', subset='validation'
)

# Load Test Data (Without Data Augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_path, target_size=(img_size, img_size),
    batch_size=batch_size, class_mode='categorical', shuffle=False
)


Found 8192 images belonging to 4 classes.
Found 2048 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load Pretrained ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze Base Model Layers
for layer in base_model.layers:
    layer.trainable = False

# Add Custom Layers
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dense(4, activation="softmax")(x)  # 4 Classes

# Create Model
model = Model(inputs=base_model.input, outputs=x)

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Model Summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 49,311,492 (188.11 MB)

 Trainable params: 25,723,780 (98.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [10]:
# Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=2,  # Increase if needed
    batch_size=batch_size
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
256/256 ━━━━━━━━━━━━━━━━━━━━ 2101s 8s/step - accuracy: 0.2507 - loss: 1.5435 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 2/2
256/256 ━━━━━━━━━━━━━━━━━━━━ 2066s 8s/step - accuracy: 0.2490 - loss: 1.3863 - val_accuracy: 0.2500 - val_loss: 1.3863


In [11]:
# Evaluate on Test Data
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc*100:.2f}%")


40/40 ━━━━━━━━━━━━━━━━━━━━ 229s 6s/step - accuracy: 0.0814 - loss: 1.3863
Test Accuracy: 35.03%


In [12]:
# Save trained model
model.save("/content/alzheimer_detection_model.h5")
print("Model saved successfully!")


Model saved successfully!
